In [1]:
import pickle
import numpy as np
import pandas as pd

In [2]:
from gryffin import Gryffin

2022-05-17 13:28:06.623615: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-17 13:28:06.623647: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
budget = 192
sampling_strategies = np.array([-1, 1])
with_desc = True
dynamic = True
random_seed = 2022

In [4]:
optimum = ['hydrazinium', 'I', 'Sn'] # value = 1.5249 eV

In [5]:
# load in the perovskites dataset
lookup_df = pickle.load(open('perovskites.pkl', 'rb'))

In [6]:
# helper functions
def measure(param):
	''' lookup the HSEO6 bandgap for given perovskite composition
	'''
	match = lookup_df.loc[
					(lookup_df.organic == param['organic']) &
					(lookup_df.anion == param['anion']) &
					(lookup_df.cation == param['cation'])
				]
	assert len(match)==1
	bandgap = match.loc[:, 'hse06'].to_numpy()[0]
	return bandgap

def get_descriptors(element, kind):
	''' retrive the descriptors for a given element
	'''
	return lookup_df.loc[(lookup_df[kind]==element)].loc[:, lookup_df.columns.str.startswith(f'{kind}-')].values[0].tolist()

In [7]:
# prepare descriptors
organic_options = lookup_df.organic.unique().tolist()
anion_options = lookup_df.anion.unique().tolist()
cation_options = lookup_df.cation.unique().tolist()

In [8]:
if with_desc:
	# use physicochemical descriptors - static or dynamic gryffin
	desc_organic = {option: get_descriptors(option, 'organic') for option in organic_options}
	desc_anion = {option: get_descriptors(option, 'anion') for option in anion_options}
	desc_cation = {option: get_descriptors(option, 'cation') for option in cation_options}
else:
	# no descriptors - naive gryffin
	desc_organic = {option: None for option in organic_options}
	desc_anion = {option: None for option in anion_options}
	desc_cation = {option: None for option in cation_options}

In [9]:
# gryffin config
config = {
	"general": {
		"num_cpus": 4,
		"auto_desc_gen": dynamic,
		"batches": 1,
		"sampling_strategies": 1,
		"boosted":  False,
		"caching": True,
		"random_seed": random_seed,
		"acquisition_optimizer": "genetic",
		"verbosity": 3
		},
	"parameters": [
		{"name": "organic", "type": "categorical", 'options': organic_options, 'category_details': desc_organic},
		{"name": "anion", "type": "categorical", 'options': anion_options, 'category_details': desc_anion},
		{"name": "cation", "type": "categorical",  'options': cation_options, 'category_details': desc_cation},
	],
	"objectives": [
		{"name": "bandgap", "goal": "min"},
	]
}

In [10]:
observations = []

# initialize gryffin
gryffin =  Gryffin(config_dict=config)

for num_iter in range(budget):

	# alternating sampling strategies, assuming batch size of 1
	idx = num_iter % len(sampling_strategies)
	sampling_strategy = sampling_strategies[idx]

	# ask Gryffin for a new sample
	samples = gryffin.recommend(observations=observations, sampling_strategies=[sampling_strategy])

	measurement = measure(samples[0])
	samples[0]['bandgap'] = measurement
	observations.extend(samples)
	print(f'ITER : {num_iter}\tSAMPLES : {samples}\t MEASUREMENT : {measurement}')


	# check for convergence
	if [samples[0]['organic'], samples[0]['anion'], samples[0]['cation']] == optimum:
		print(f'FOUND OPTIMUM AFTER {num_iter+1} ITERATIONS!')
		break


Could not find any observations, falling back to random sampling

Memory usage: 0 GB, 389 MB, 628 kB

Overall time required: 24.3 ms

ITER : 0	SAMPLES : [{'organic': 'dimethylammonium', 'anion': 'Cl', 'cation': 'Pb', 'bandgap': 3.3139}]	 MEASUREMENT : 3.3139


1 observations found

...

Process Process-16:
Process Process-18:
Traceback (most recent call last):
  File "/ssd003/home/jwilles/condaenvs/gryffin-sphinx/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/ssd003/home/jwilles/condaenvs/gryffin-sphinx/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/ssd003/home/jwilles/condaenvs/gryffin-sphinx/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/ssd003/home/jwilles/condaenvs/gryffin-sphinx/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/ssd003/home/jwilles/condaenvs/gryffin-sphinx/lib/python3.8/site-packages/gryffin/acquisition/acquisition.py", line 171, in _proposal_optimization_thread
    local_optimizer = self._load_optimizer(acquisition_constraints=acquisition_constraints)
  File "/ssd003/home/jwilles/condaenvs/gryff

package "deap" required by GeneticOptimizer not found, please install

  File "/ssd003/home/jwilles/condaenvs/gryffin-sphinx/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/ssd003/home/jwilles/condaenvs/gryffin-sphinx/lib/python3.8/site-packages/gryffin/acquisition/acquisition.py", line 171, in _proposal_optimization_thread
    local_optimizer = self._load_optimizer(acquisition_constraints=acquisition_constraints)
  File "/ssd003/home/jwilles/condaenvs/gryffin-sphinx/lib/python3.8/site-packages/gryffin/acquisition/acquisition.py", line 356, in _load_optimizer
    from .genetic_optimizer import GeneticOptimizer
  File "/ssd003/home/jwilles/condaenvs/gryffin-sphinx/lib/python3.8/site-packages/gryffin/acquisition/genetic_optimizer/__init__.py", line 8, in <module>
    from .genetic_optimizer import GeneticOptimizer
  File "/ssd003/home/jwilles/condaenvs/gryffin-sphinx/lib/python3.8/site-packages/gryffin/acquisition/genetic_optimizer/genetic_optimizer.py", line 9, in <module>
    from deap imp

Process Process-19:
  File "/ssd003/home/jwilles/condaenvs/gryffin-sphinx/lib/python3.8/site-packages/gryffin/acquisition/acquisition.py", line 356, in _load_optimizer
    from .genetic_optimizer import GeneticOptimizer
Traceback (most recent call last):
  File "/ssd003/home/jwilles/condaenvs/gryffin-sphinx/lib/python3.8/site-packages/gryffin/acquisition/genetic_optimizer/__init__.py", line 8, in <module>
    from .genetic_optimizer import GeneticOptimizer
  File "/ssd003/home/jwilles/condaenvs/gryffin-sphinx/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/ssd003/home/jwilles/condaenvs/gryffin-sphinx/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/ssd003/home/jwilles/condaenvs/gryffin-sphinx/lib/python3.8/site-packages/gryffin/acquisition/genetic_optimizer/genetic_optimizer.py", line 9, in <module>
    from deap import base, creator, tools
ModuleNotFoundError: No module named '

KeyError: 0